In [2]:
# !pip install python-dotenv

In [3]:
import requests
import base64
import pandas as pd
import numpy as np
import pytz
import os
from dotenv import load_dotenv

In [4]:
load_dotenv()

organization = os.getenv("ORGANIZATION")
project = os.getenv("PROJECT")
pat = os.getenv("AZURE_PAT")


credentials = base64.b64encode(f":{pat}".encode("utf-8")).decode("utf-8")

url = f"https://dev.azure.com/{organization}/{project}/_apis/wit/reporting/workitemrevisions?includeLatestOnly=false&includeDeleted=false&startDateTime=2024-10-01T00:00:00Z&api-version=7.1"

params = {
    "$expand": "none",
    "$skip": 0, 
    "$top": 100 
}

headers = {
    "Authorization": f"Basic {credentials}",
    "Content-Type": "application/json",
}

In [5]:
def fetch_all_work_items():
    all_work_items = []
    continuation_token = None

    while True:
        if continuation_token:
            params["continuationToken"] = continuation_token

        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            data = response.json()
            all_work_items.extend(data.get("values", []))
            
            if not len(data.get('values', [])):
                break
            continuation_token = data.get("continuationToken") 

        else:
            print("Failed to fetch work items. Status code:", response.status_code)
            print("Response:", response.text)
            break
    
    
    return all_work_items


In [6]:
work_items = fetch_all_work_items()
print(f"Total records fetched: {len(work_items)}")


Total records fetched: 24332


In [7]:
data = [
    {**item["fields"]}
    for item in work_items
]

df = pd.DataFrame(data)

In [8]:
india_timezone = pytz.timezone('Asia/Kolkata')

def convert_to_ist(date):
    if pd.isna(date):
        return date
    elif date.tzinfo is None: 
        return date.tz_localize('UTC').tz_convert(india_timezone)
    else:
        return date.tz_convert(india_timezone)


df['System.CreatedDate'] = pd.to_datetime(df['System.CreatedDate'], errors='coerce').apply(convert_to_ist)
df['Microsoft.VSTS.Scheduling.StartDate'] = pd.to_datetime(df['Microsoft.VSTS.Scheduling.StartDate'], errors='coerce').apply(convert_to_ist)
df['Microsoft.VSTS.Scheduling.TargetDate'] = pd.to_datetime(df['Microsoft.VSTS.Scheduling.TargetDate'], errors='coerce').apply(convert_to_ist)
df['System.ChangedDate'] = pd.to_datetime(df['System.ChangedDate'], errors='coerce').apply(convert_to_ist)
df['Microsoft.VSTS.Common.ClosedDate'] = pd.to_datetime(df['Microsoft.VSTS.Common.ClosedDate'], errors='coerce').apply(convert_to_ist)


In [9]:
df = df.rename(columns={
    "System.Id": "ID",
    "System.Rev": "Revision",
    "System.AreaId": "AreaId",
    "System.AreaPath": "AreaPath",
    "System.CreatedDate": "CreatedDate",
    "System.CreatedBy": "CreatedBy",
    "Microsoft.VSTS.Scheduling.StartDate": "StartDate",
    "Microsoft.VSTS.Scheduling.TargetDate": "TargetDate",
    "System.ChangedDate": "ChangedDate",
    "System.ChangedBy": "ChangedBy",
    "System.Reason": "Reason",
    "System.State": "State",
    "System.WorkItemType": "WorkItemType",
    "System.AssignedTo": "AssignedTo",
    "System.Title": "Title",
    "Microsoft.VSTS.Scheduling.RemainingWork": "RemainingWork",
    "Microsoft.VSTS.Scheduling.OriginalEstimate": "OriginalEstimate",
    "Microsoft.VSTS.Scheduling.CompletedWork": "CompletedWork",
    "Microsoft.VSTS.Common.ClosedDate": "ClosedDate",
    "Microsoft.VSTS.Common.ClosedBy": "ClosedBy",
    "System.Tags": "Tags",  
    "Microsoft.VSTS.Common.Priority": "Priority", 
    "Custom.PriorityI": "PriorityI", 
})


In [10]:
desired_columns = [
    "ID", "Revision", "AreaId", "AreaPath", "CreatedDate", "CreatedBy",
    "StartDate", "TargetDate", "ChangedDate", "ChangedBy", "Reason", "State",
    "WorkItemType", "AssignedTo", "Title", "RemainingWork", "OriginalEstimate",
    "CompletedWork", "ClosedDate", "ClosedBy", "Tags", "Priority", "PriorityI"
]

df = df[desired_columns]

In [11]:
df['AssignedTo'] = df['AssignedTo'].str.extract(r'(.*?)(?=<)')

In [12]:
def extract_project_name(value):
    if value == 'managed-services':
        return value
    return value.split('\\')[-1]

df['Project Name'] = df['AreaPath'].apply(extract_project_name)

In [13]:
df['WorkHours'] = None
df['Date'] = None

In [14]:
df['WorkHours'] = df['WorkHours'].astype(float)
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

In [15]:
df['Date'] = np.where(
    (df['StartDate'].isnull()) & (df['TargetDate'].isnull()), 
    df['ChangedDate'].dt.date,  # If both are null, use ChangedDate
    np.where(
        df['TargetDate'].isnull(),
        np.where(
            df['StartDate'] > df['ChangedDate'], 
            df['StartDate'].dt.date,  # If StartDate > ChangedDate, use StartDate
            np.where(
                df['State'] == 'Doing', 
                df['ChangedDate'].dt.date,  # If State == 'doing', use ChangedDate
                df['StartDate'].dt.date     # Otherwise, use StartDate
            )
        ),
        np.where(
            df['StartDate'].isnull(), 
            np.where(
                df['TargetDate'] >= df['ChangedDate'],
                df['ChangedDate'].dt.date,  # If TargetDate >= ChangedDate, use ChangedDate
                df['TargetDate'].dt.date    # Otherwise, use TargetDate
            ),
            np.where(
                (df['ChangedDate'] >= df['StartDate']) & (df['ChangedDate'] <= df['TargetDate']),
                df['ChangedDate'].dt.date,  # If ChangedDate is between StartDate and TargetDate, use ChangedDate
                np.where(
                    (df['ChangedDate'] <= df['StartDate']) & (df['ChangedDate'] <= df['TargetDate']),
                    df['StartDate'].dt.date,  # If ChangedDate is <= both StartDate and TargetDate, use StartDate
                    df['TargetDate'].dt.date    # Otherwise, use TargetDate
                )
            )
        )
    )
)


In [16]:
df = df.sort_values(by=["ID", "ChangedDate"]).reset_index(drop=True)

for id_group, group in df.groupby('ID'):
    if group.empty:
        continue  # Skip empty groups
    prev_completed_work = None
    for i, row in group.iterrows():
        current_completed_work = row['CompletedWork']
        if pd.isna(prev_completed_work):
            if not pd.isna(current_completed_work):
                df.at[i, 'WorkHours'] = current_completed_work
            else:
                df.at[i, 'WorkHours'] = np.nan
        else:
            if not pd.isna(current_completed_work):
                df.at[i, 'WorkHours'] = current_completed_work - prev_completed_work
            else:
                df.at[i, 'WorkHours'] = np.nan
        prev_completed_work = current_completed_work

In [17]:
df = df.sort_values(by=['ID', 'Date', 'ChangedDate'], ascending=[True, True, True])

# Perform the groupby operation, aggregating the relevant columns
grouped_df = df.groupby(['ID', 'Date'], as_index=False).agg(
    {
        # For each column, take the latest value based on ChangedDate (last within the group)
        'Revision': 'last',
        'AreaId': 'last',
        'AreaPath': 'last',
        'CreatedDate': 'last',
        'CreatedBy': 'last',
        'StartDate': 'last',
        'TargetDate': 'last',
        'ChangedDate': 'last',
        'ChangedBy': 'last',
        'Reason': 'last',
        'State': 'last',
        'WorkItemType': 'last',
        'AssignedTo': 'last',
        'Title': 'last',
        'RemainingWork': 'last',
        'OriginalEstimate': 'last',
        'CompletedWork': 'last',
        'ClosedDate': 'last',
        'ClosedBy': 'last',
        'Tags': 'last',
        'Priority': 'last',
        'PriorityI': 'last',
        # 'WorkPriority': 'last',
        # 'PriorityD': 'last',
        'Project Name': 'last',
        
        # Sum the WorkHours within the group
        'WorkHours': 'sum'
    }
)



In [18]:
grouped_df.to_csv('output_file.csv', index=False)